In [1]:
!git clone -b implementing-vars https://github.com/DrYogurt/Aero-Design-Team-Gamma.git
!pip install poetry==1.4.2

Cloning into 'Aero-Design-Team-Gamma'...
remote: Enumerating objects: 206, done.
remote: Counting objects: 100% (206/206), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 206 (delta 87), reused 155 (delta 47), pack-reused 0 (from 0)
Receiving objects: 100% (206/206), 489.49 KiB | 1.97 MiB/s, done.
Resolving deltas: 100% (87/87), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of poetry-plugin-export to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of virtualenv to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of virtualenv to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter

In [1]:
%cd Aero-Design-Team-Gamma
!poetry config virtualenvs.in-project true
!poetry install --no-ansi

/content/Aero-Design-Team-Gamma
Creating virtualenv aerospace-design-team-gamma in /content/Aero-Design-Team-Gamma/.venv
Installing dependencies from lock file
The lock file might not be compatible with the current version of Poetry.
Upgrade Poetry to ensure the lock file is read properly or, alternatively, regenerate the lock file with the `poetry lock` command.

Package operations: 41 installs, 0 updates, 0 removals

  • Installing numpy (1.26.4)
  • Installing six (1.17.0)
  • Installing contourpy (1.3.1)
  • Installing cycler (0.12.1)
  • Installing fonttools (4.55.3)
  • Installing kiwisolver (1.4.8)
  • Installing packaging (24.2)
  • Installing pillow (11.1.0)
  • Installing pyparsing (3.2.1)
  • Installing python-dateutil (2.9.0.post0)
  • Installing click (8.1.8)
  • Installing iniconfig (2.0.0)
  • Installing mccabe (0.7.0)
  • Installing matplotlib (3.10.0)
  • Installing markupsafe (3.0.2)
  • Installing mpmath (1.3.0)
  • Installing mypy-extensions (1.0.0)
  • Installing p

In [2]:
VENV_PATH = "/content/Aero-Design-Team-Gamma/.venv/lib/python3.11/site-packages"
import os, sys
LOCAL_VENV_PATH = '/content/venv' # local notebook
os.symlink(VENV_PATH, LOCAL_VENV_PATH) # connect to directory in drive
sys.path.insert(0, LOCAL_VENV_PATH)

In [32]:
from ambiance import Atmosphere

from global_variables.solver import EquationSystem
from global_variables.registry import VariableRegistry, Variable

registry = VariableRegistry("aero_vars.yaml")

In [33]:
#outputs = {"S_TO", "R", "sigma_max"}
outputs = {"R", "sigma_max"}

# Create solvers for each output
solvers = {
    output: EquationSystem(registry, {output})
    for output in outputs
}

# Get all unique inputs
all_inputs = set().union(*(solver.inputs for solver in solvers.values()))
print(f"Required inputs: {all_inputs}")
solvers

Required inputs: {'n_pax', 'W_max', 'V', 'b', 'W_pax', 'S_', 'rho', 'T_A0', 'TSFC', 'C_D0', 'e', 'We_Wmax'}


{'sigma_max': <global_variables.solver.EquationSystem at 0x7b3b7fdd3ed0>,
 'R': <global_variables.solver.EquationSystem at 0x7b3ba0f16060>}

In [34]:
def mach(M,h):
  atmo = Atmosphere(h * 0.3048)
  return atmo.speed_of_sound[-1] * M / 0.3048

In [35]:
def rho_func(h):
  atmo = Atmosphere(h * 0.3048)
  return atmo.density[-1]

In [36]:

# Generate sample points for each input
n_samples = int(1e2)
samples = {}

# Define ranges for each input (you'll need to adjust these)
ranges = {
    'V':(mach(0.8,35e3),mach(0.95,35e3)), #cruising velocity ft / s
    'S_': (8e4,11e4), # area in ft^2
    'b': (300, 315),  # wingspan in ft^2
    'T_A0': (4.8e5,5.2e5),
    'W_pax': (200,300), #weight per passenger in lbs
    'e':(0.8,1.), # oswald efficiency
    'mu':(.01,.05),
    'W_max':(1.9e6, 2.2e6),  # weight in lbs
    'C_Lmax':(1.5,1.8),  # weight in lbs
    'C_D0':(0.01,0.035),
    'rho':(rho_func(45e3),rho_func(0)),
    'n_pax': (1200,1260), # number of passengers
    'We_Wmax':(0.4,0.6),
    #'h': (0, 40000),  # altitude in ft
    'TSFC': (0.3*3600.,0.6*3600), # TSFC

    # Add other inputs and their ranges
}
print(ranges)

{'V': (778.514759070209, 924.4862763958732), 'S_': (80000.0, 110000.0), 'b': (300, 315), 'T_A0': (480000.0, 520000.0), 'W_pax': (200, 300), 'e': (0.8, 1.0), 'mu': (0.01, 0.05), 'W_max': (1900000.0, 2200000.0), 'C_Lmax': (1.5, 1.8), 'C_D0': (0.01, 0.035), 'rho': (0.23824523567640446, 1.225000018124288), 'n_pax': (1200, 1260), 'We_Wmax': (0.4, 0.6), 'TSFC': (1080.0, 2160.0)}


In [37]:
import numpy as np
import matplotlib.pyplot as plt
# Generate Latin Hypercube samples
from scipy.stats import qmc
sampler = qmc.LatinHypercube(d=len(all_inputs))
sample_points = sampler.random(n=n_samples)

# Scale samples to actual ranges
input_list = sorted(all_inputs)
for i, input_var in enumerate(input_list):
    if input_var in ranges:
        min_val, max_val = ranges[input_var]
        samples[input_var] = qmc.scale(sample_points[:, [i]], [min_val], [max_val]).flatten()
    else:
        print(f"Warning: No range defined for {input_var}")
        samples[input_var] = sample_points[:, i]  # Use 0-1 range as default

In [38]:
# Calculate outputs
results = {}
for output, solver in solvers.items():
    solver_f,_ = solver.create_solver()
    solver_func = solver_f[output]
    input_args = [samples[inp] for inp in sorted(solver.inputs)]
    try:
        output_values = np.array(solver_func(*input_args))
        print(output_values)
        # Ensure output is 1D array of correct length
        if output_values.size == 1:
            output_values = np.full(n_samples, output_values)
        results[output] = output_values
    except Exception as e:
        print(f"Error calculating {output}: {e}")
#results

[0.87328054 0.7352643  0.78240292 0.81881931 0.61706671 0.89149786
 0.61716097 0.65284904 0.68134908 0.98651618 0.63589292 0.78318423
 0.92470549 0.72025485 0.80724701 0.44766982 0.44098039 0.89720381
 0.8963806  0.75450868 0.8842429  0.53357313 0.78101291 1.00951195
 0.68661565 0.57486476 0.89408277 0.61131255 0.68619724 0.83189647
 0.69525497 0.52970882 0.85120545 0.52817174 0.72488166 0.821674
 0.92077028 0.96071649 0.74619566 0.69718423 0.7095313  0.71656011
 0.7339755  0.67614134 0.78899269 0.45976996 0.78970062 0.59449238
 0.82324261 0.70300492 0.91670586 0.67657972 0.75650903 0.60810262
 0.85715008 0.60178618 0.89469275 0.82313954 0.76899691 0.57900315
 0.86440114 0.7956806  0.71491657 0.66442421 0.64606972 0.70803596
 0.5526146  0.57517321 0.72928824 0.49472482 0.79832731 0.60266887
 0.74933905 0.70310537 0.71899775 0.68556631 0.56898477 0.67424007
 0.79641968 0.75757033 0.63877746 0.65759071 0.50024959 0.58215267
 0.78484861 0.95307788 0.79954228 0.69144014 0.86611885 0.721412

In [44]:
def create_output_corner_plot(samples, output_values, setpoint, output_name, 
                            figsize=(10, 10), n_contours=20, n_grid=50):
    """
    Create a corner plot with contours showing how output varies with input pairs.
    """
    # Print value ranges for debugging
    print(f"Output range: {np.min(output_values):.2f} to {np.max(output_values):.2f}")
    print(f"Setpoint: {setpoint}")
    
    # Create diverging colormap centered at setpoint
    colors = ['darkred', 'red', 'lightcoral', 'white', 'lightgreen', 'green', 'darkgreen']
    custom_cmap = LinearSegmentedColormap.from_list('custom', colors, N=256)
    
    input_vars = list(samples.keys())
    n_vars = len(input_vars)
    
    # Create figure with small margins
    fig, axes = plt.subplots(n_vars, n_vars, figsize=figsize)
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    
    # If only one variable, axes won't be a 2D array
    if n_vars == 1:
        axes = np.array([[axes]])
    
    # Calculate differences and normalize
    differences = output_values - setpoint
    abs_max = max(abs(np.min(differences)), abs(np.max(differences)))
    
    # Create levels based on percentage differences from setpoint
    rel_diffs = differences / abs(setpoint) * 100  # Convert to percentage
    max_rel_diff = max(abs(np.min(rel_diffs)), abs(np.max(rel_diffs)))
    
    # Create symmetric levels around zero in percentage terms
    level_step = max_rel_diff / (n_contours // 2)
    levels = np.concatenate([
        np.arange(-max_rel_diff, 0, level_step),
        np.array([0]),
        np.arange(level_step, max_rel_diff + level_step, level_step)
    ])
    
    # Create the corner plot
    for i in range(n_vars):
        for j in range(n_vars):
            ax = axes[i, j]
            
            # Remove top and right spines for all plots
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            
            if i < j:  # Upper triangle - leave empty
                ax.axis('off')
                continue
                
            elif i == j:  # Diagonal - histogram
                ax.hist(samples[input_vars[i]], bins=30, density=True)
                if i == n_vars-1:  # Only bottom row gets x labels
                    ax.set_xlabel(input_vars[i])
                else:
                    ax.set_xticklabels([])
                
                if j == 0:  # Only first column gets y labels
                    ax.set_ylabel('Density')
                else:
                    ax.set_yticklabels([])
                
            else:  # Lower triangle - contour plots
                x = samples[input_vars[j]]
                y = samples[input_vars[i]]
                
                # Create grid for contour plot
                xi = np.linspace(x.min(), x.max(), n_grid)
                yi = np.linspace(y.min(), y.max(), n_grid)
                xi, yi = np.meshgrid(xi, yi)
                
                # Interpolate relative differences on the grid
                zi = griddata((x, y), rel_diffs, (xi, yi), method='cubic')
                
                # Create contour plot
                contour = ax.contourf(xi, yi, zi, levels=levels, cmap=custom_cmap, extend='both')
                
                # Add colorbar only for rightmost plots
                if j == n_vars-1:
                    cbar = plt.colorbar(contour, ax=ax, ticks=levels[::2])
                    cbar.set_label('% Difference from setpoint')
                
                # Add setpoint contour in black
                ax.contour(xi, yi, zi, levels=[0], colors='black', linestyles='dashed')
                
                # Only add x labels to bottom row
                if i == n_vars-1:
                    ax.set_xlabel(input_vars[j])
                else:
                    ax.set_xticklabels([])
                
                # Only add y labels to leftmost column
                if j == 0:
                    ax.set_ylabel(input_vars[i])
                else:
                    ax.set_yticklabels([])
    
    plt.suptitle(f'Parameter Effects on {output_name} (Setpoint: {setpoint})', y=1.02)
    return fig, axes

In [45]:
setpoints={'R':1e4*6076,'sigma_max':0.1852}
for output in solvers.keys():
    fig, axes = create_output_corner_plot(samples,results[output],output_name = output, setpoint=setpoints[output],
                                         figsize=(30,30))
    # Add title
    #fig.suptitle(f"Corner Plots for {output}", fontsize=14, y=1.02)
    
    # Save figure
    plt.savefig(f"corner_plot_{output}.png", dpi=300)
    plt.close()

Output range: 0.44 to 1.01
Setpoint: 0.1852
Output range: 0.00 to 0.01
Setpoint: 60760000.0


In [ ]:
R_func, soln = solvers['S_TO'].create_solver()

soln
#from sympy import lambdify
#func = lambdify(soln


In [12]:
solvers['R']._collect_equations()

{'C_L_full': ['2 * W_max / (rho * V**2 * S_)'],
 'CL_CD_ratio': ['0.5 * ((C_L_full)**(1/2)/C_D_full + (C_L_empty)**(1/2)/C_D_empty)'],
 'W_e': ['We_Wmax * W_max'],
 'C_L_empty': ['2 * (W_max - W_f) / (rho * V**2 * S_)'],
 'AR': ['b**2 / S_'],
 'C_D_empty': ['C_D0 + C_Di_empty'],
 'W_p': ['n_pax * W_pax'],
 'C_D_full': ['C_D0 + C_Di_full'],
 'C_Di_empty': ['C_L_empty**2 / (pi * e * AR)'],
 'R': ['(2/TSFC) * (2/(rho*S_)**(1/2)) * CL_CD_ratio * ((W_max)**(1/2)-(W_max-W_f)**(1/2))'],
 'C_Di_full': ['C_L_full**2 / (pi * e * AR)'],
 'W_f': ['W_max - W_p - W_e']}

In [ ]:
3.0e-14
*W_max
*(-5.04e+31*C_D0*C_Lmax*S_*W_max*b**2*e*rho
  + 1.28899111083484e+29*C_D0*W_max**3*b**2*e
  *log(25.0*C_Lmax*(-T_A0 + W_max*mu)/(72.0*C_D0*W_max**2 - 25.0*C_Lmax*T_A0))
  + 4.94669432124992e+27*C_Lmax**2*S_*W_max*log(25.0*C_Lmax*(-T_A0 + W_max*mu)/(72.0*C_D0*W_max**2 - 25.0*C_Lmax*T_A0)) + 1.75e+31*C_Lmax**2*S_*b**2*e*mu*rho - 4.47566357928762e+28*C_Lmax*T_A0*W_max*b**2*e*log(25.0*C_Lmax*(-T_A0 + W_max*mu)/(72.0*C_D0*W_max**2 - 25.0*C_Lmax*T_A0)))/(S_*rho*(1.24416000000001e+17*C_D0**2*W_max**3*b**2*e + 4.77464829275686e+15*C_D0*C_Lmax**2*S_*W_max - 4.32e+16*C_D0*C_Lmax*T_A0*W_max*b**2*e - 4.32000000000002e+16*C_D0*C_Lmax*W_max**2*b**2*e*mu - 1.65786399054058e+15*C_Lmax**3*S_*mu + 1.5e+16*C_Lmax**2*T_A0*b**2*e*mu))